https://developer.here.com/blog/getting-started-with-geocoding-exif-image-metadata-in-python3

https://reverse.geocoder.api.here.com/6.2/reversegeocode.json?prox=37.7442%2C-119.5931%2C1000&mode=
#retrieveAreas&app_id=devportal-demo-20180625&app_code=9v2BkviRwi9Ot26kp2IysQ&gen=9

In [161]:
# define functions to get geodata data . I got the instrutions from a website
import os
import requests
import json

from PIL import Image

# funtion 1
def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    return image._getexif()



from PIL.ExifTags import GPSTAGS
from PIL.ExifTags import TAGS
# funtion #2
def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")

    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    return geotagging


#funtion 3
def get_decimal_from_dms(dms, ref):

    degrees = dms[0][0] / dms[0][1]
    minutes = dms[1][0] / dms[1][1] / 60.0
    seconds = dms[2][0] / dms[2][1] / 3600.0

    if ref in ['S', 'W']:
        degrees = -degrees
        minutes = -minutes
        seconds = -seconds

    return round(degrees + minutes + seconds, 5)


# funtion 4

def get_coordinates(geotags):
    lat = get_decimal_from_dms(geotags['GPSLatitude'], geotags['GPSLatitudeRef'])

    lon = get_decimal_from_dms(geotags['GPSLongitude'], geotags['GPSLongitudeRef'])

    return (lat,lon)


# Funtion 5

def get_location(geotags):
    
    
    coords = get_coordinates(geotags)

    uri = 'https://reverse.geocoder.api.here.com/6.2/reversegeocode.json'
    headers = {}
    params = {
        'app_id': 'cjIyVcZ1rzUhd0Xb9r0q',
        'app_code': 'f9pU60MvwiOVXVRl5N5HmA',
        'prox': "%s,%s" % coords,
        'gen': 9,
        'mode': 'retrieveAddresses',
        'maxresults': 1,
    }

    response = requests.get(uri, headers=headers, params=params)
    try:
        response.raise_for_status()
        return response.json()

    except requests.exceptions.HTTPError as e:
        print(str(e))
        return {}


# Funtcion 5 modified to allow for no geotag data

def get_location2(geotags):
    if "GPSLatitude" in geotags:
        
        coords = get_coordinates(geotags)
        uri = 'https://reverse.geocoder.api.here.com/6.2/reversegeocode.json'
        headers = {}
        params = {
            'app_id': 'cjIyVcZ1rzUhd0Xb9r0q',
            'app_code': 'f9pU60MvwiOVXVRl5N5HmA',
            'prox': "%s,%s" % coords,
            'gen': 9,
            'mode': 'retrieveAddresses',
            'maxresults': 1,
        }

        response = requests.get(uri, headers=headers, params=params)
        return response.json()
    
    else: 
        
        response={'Response': { 'View':[{ 'Result': [{'Location': {'MapReference':{ 'MapReleaseDate':''}, 'DisplayPosition': {'Latitude': 0, 'Longitude': 0},'MapView': {'TopLeft': {'Latitude': 38.90021, 'Longitude': -77.05679},'BottomRight': {'Latitude': 38.88508, 'Longitude': -77.00202}},'Address': {'Label': 'Washington Mall, Washington, DC, United States','Country': 'USA','State': 'DC','County': 'District of Columbia','City': 'Washington','District': 'Washington Mall','PostalCode': '20004'}}}]}]} }
        return response




testing

In [167]:

exif = get_exif("static/Images/uploads/smash_4296.JPG")
# smash_4296 NoGeoTag
#exif

geotags = get_geotagging(exif)
geotags
#coord=get_coordinates(geotags)
#coord
#location = get_location(geotags)
#location
test=get_location2(geotags)
test['Response']
#test


{'MetaInfo': {'Timestamp': '2020-03-01T21:57:05.930+0000'},
 'View': [{'_type': 'SearchResultsViewType',
   'ViewId': 0,
   'Result': [{'Relevance': 1.0,
     'Distance': -138.8,
     'Direction': 108.4,
     'MatchLevel': 'district',
     'MatchQuality': {'Country': 1.0,
      'State': 1.0,
      'County': 1.0,
      'City': 1.0,
      'District': 1.0,
      'PostalCode': 1.0},
     'Location': {'LocationId': 'NT_zeyJ5M16Ci53y9KE8RMTGD',
      'LocationType': 'area',
      'DisplayPosition': {'Latitude': 38.89142, 'Longitude': -77.03367},
      'MapView': {'TopLeft': {'Latitude': 38.90021, 'Longitude': -77.05679},
       'BottomRight': {'Latitude': 38.88508, 'Longitude': -77.00202}},
      'Address': {'Label': 'Washington Mall, Washington, DC, United States',
       'Country': 'USA',
       'State': 'DC',
       'County': 'District of Columbia',
       'City': 'Washington',
       'District': 'Washington Mall',
       'PostalCode': '20004',
       'AdditionalData': [{'value': 'United 

In [137]:
# create a function to put the geotag data into a dictionary

def ExtractPicData(picAddress):
    PicData={'latitude':"",'longitude':'','landmark':'','country':'','state':'','county':'','city':'','zipcode':'','ImageTimeStamp':'','FileAddress':''}
    exif = get_exif(picAddress)
    geotags = get_geotagging(exif)
    location = get_location2(geotags)

    latitude=location['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Latitude']
    longitude=location['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Longitude']
    landmark=location['Response']['View'][0]['Result'][0]['Location']['Address']['Label']
    country=location['Response']['View'][0]['Result'][0]['Location']['Address']['Country']
    state=location['Response']['View'][0]['Result'][0]['Location']['Address']['State']
    county=location['Response']['View'][0]['Result'][0]['Location']['Address']['County']
    city=location['Response']['View'][0]['Result'][0]['Location']['Address']['City']
    zipcode=location['Response']['View'][0]['Result'][0]['Location']['Address']['PostalCode']
    ImageTimeStamp=location['Response']['View'][0]['Result'][0]['Location']['MapReference']['MapReleaseDate']
    FileAddress=picAddress

    PicData['latitude']=latitude
    PicData['longitude']=longitude
    PicData['landmark']=landmark
    PicData['country']=country
    PicData['state']=state
    PicData['county']=county
    PicData['city']=city
    PicData['zipcode']=zipcode
    PicData['ImageTimeStamp']=ImageTimeStamp
    PicData['FileAddress']="../"+FileAddress
    return PicData


In [109]:
# execute function

uploadsFolder="static/Images/uploads/"
picAddress=uploadsFolder +'NoGeoTag.JPG'
ReturnedData=ExtractPicData(picAddress)
# ReturnedData=PicData

# DataFrame=[]
# DataFrame.append(ReturnedData)

ReturnedData

{'latitude': 0,
 'longitude': 0,
 'landmark': 'Washington Mall, Washington, DC, United States',
 'country': 'USA',
 'state': 'DC',
 'county': 'District of Columbia',
 'city': 'Washington',
 'zipcode': '20004',
 'ImageTimeStamp': '',
 'FileAddress': '../static/Images/uploads/NoGeoTag.JPG'}

In [166]:
# get all the files in the images folder into a list called dirs

import os, sys
import pandas as pd
path = "static/images/uploads/"
dirs = os.listdir(path )

# Create an empty dictionary
PicturesDictionary=[]
# for each file in dirs execute extractPicData, check that the file is a jpg

for file in dirs:
    print(file)
    if file.find('JPG') is not -1:
        Picdata=path+file
        # Eexecute the function and insert the data retured by the function into a variable 
        ReturnedData=ExtractPicData(Picdata)
        
        # Append the data into the ditionary                 
        PicturesDictionary.append(ReturnedData)
    if file.find('jpg') is not -1:
        Picdata=path+file
        # Eexecute the function and insert the data retured by the function into a variable 
        ReturnedData=ExtractPicData(Picdata)
        
        # Append the data into the ditionary                 
        PicturesDictionary.append(ReturnedData)
    

# Convert the dictionary into a dataframe
PicturesDataFrame=pd.DataFrame(PicturesDictionary)   
#PicturesDictionary
PicturesDataFrame.tail(1)
#print(dirs)

crumple_4188.JPG
metal_4265.JPG
Nogeotag
NoGeoTag.JPG
partial_4172.jpg
smash_2532.JPG
smash_4128.JPG
smash_4296.JPG
stand_4174.JPG
Thumbs.db


,FileAddress,ImageTimeStamp,city,country,county,landmark,latitude,longitude,state,zipcode
7,../static/images/uploads/stand_4174.JPG,2020-01-31,St Paul,USA,Ramsey,"Summit-University, St Paul, MN, United States",44.95129,-93.12664,MN,55104


In [165]:
PicturesDataFrame.columns

Index(['FileAddress', 'ImageTimeStamp', 'city', 'country', 'county',
       'landmark', 'latitude', 'longitude', 'state', 'zipcode'],
      dtype='object')

In [147]:
PicturesDataFrame.tail(1)

,FileAddress,ImageTimeStamp,city,country,county,landmark,latitude,longitude,state,zipcode
6,../static/images/uploads/stand_4174.JPG,2020-01-31,St Paul,USA,Ramsey,"Summit-University, St Paul, MN, United States",44.95129,-93.12664,MN,55104


In [8]:
# save to csv
# PicturesDataFrame.to_csv("db/OldportsGeoTagData.csv", index=True)
PicturesDataFrame.to_csv("db/OldportsGeoTagData.csv", index=False)



move and shrink file

In [188]:
def make_thumbnail(filename):
    img = Image.open(filename)
    img.thumbnail((400,400))

    (width, height) = img.size
    if width > height:
        upImage = img.rotate(90, expand=True)
        
    else:
        upImage = img

    upImage.thumbnail((400,400))
#    img.save(filename)
    upImage.save('static/Images/uploads/testrotate.jpg')


In [189]:
make_thumbnail("static/Images/uploads/smash_4128.JPG")

make_thumbnail('images/20190704_210738_Film3.jpg')

In [22]:

import os, sys
import pandas as pd
path = "static/images/"
dirs = os.listdir(path )

# Create an empty dictionary
PicturesDictionary=[]
# for each file in dirs execute extractPicData, check that the file is a jpg

for file in dirs:
    print(file)
    if file.find('jpg') is not -1:
        Picdata=path+file
        make_thumbnail(Picdata)
        # Eexecute the function and insert the data retured by the function into a variable 
       

away_4732.JPG
away_5731.JPG
away_6258.JPG
crumple_254.JPG
crumple_4188.JPG
crumple_458.JPG
crumple_5427.JPG
crumple_6198.JPG
fun_1287.jpg
fun_3905.JPG
fun_4048.JPG
fun_4356.JPG
fun_808.JPG
fun_842.JPG
gold_3710.JPG
gold_6480.JPG
IMG_3416.jpg
leaves_1325.JPG
leaves_1331.JPG
leaves_1539.JPG
leaves_2531.JPG
leaves_2784.JPG
leaves_3430.JPG
leaves_3731.JPG
leaves_3761.JPG
leaves_3763.JPG
leaves_3764.JPG
leaves_3820.JPG
leaves_4610.JPG
leaves_4656.JPG
leaves_4688.JPG
leaves_4689.JPG
leaves_4798.JPG
leaves_5191.JPG
leaves_5423.JPG
leaves_5737.JPG
leaves_5809.JPG
leaves_5849.JPG
leaves_5989.JPG
leaves_6013.JPG
leaves_6080.JPG
leaves_6121.JPG
leaves_6299.JPG
leaves_6459.JPG
leaves_6464.JPG
leaves_6472.JPG
leaves_6478.JPG
leaves_6479.JPG
leaves_6717.JPG
leaves_6718.JPG
leaves_8001.JPG
lush_1482.JPG
lush_1541.JPG
lush_1636.JPG
lush_1669.JPG
lush_3996.JPG
lush_6457.JPG
lush_7312.JPG
metal_1388.JPG
metal_1401.JPG
metal_3186.JPG
metal_4265.JPG
metal_5158.JPG
metal_6206.JPG
metal_7569.JPG
metal_8440.